In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import time
import hashlib
from neo4j import GraphDatabase
import json
import re


In [2]:
db_host = "bolt://localhost:7687"
password = "1234qwer"


In [3]:
api_characters = pd.read_csv("./Datasets/api_character_list.csv")
scraping_characters = pd.read_csv(
    "./Datasets/scraping_personaggi_completo_filtered_relations.csv")


In [ ]:
scraping_characters.head(20)

In [ ]:
api_characters.head(20)

Add characters nodes with variant from API

In [50]:
def add_character_node(tx, name, character_id, character_description):
  processed_name = name.replace('"', " ").replace('"'," ")
  nome_senza_parentesi = re.sub("[\(\[].*?[\)\]]", "", processed_name.strip())
  query_string = 'MERGE (c:character{name: "%s"})' %(nome_senza_parentesi.replace("'", " ").lower().strip())
  tx.run(query_string)
  tx.run(
      "MERGE (n:character_variant {name: '%s', character_id: %s, character_description: '%s'})" %(name.replace("'", ' ').replace('"', " ").replace("-", " "), character_id, str(character_description).replace("'", ' ').replace('"', " ").replace("-", " ")))
  query_variant = 'MATCH (a:character{name:"%s"}),(b:character_variant {name: "%s", character_id: %s, character_description: "%s"}) MERGE (b)-[r:variante_di]->(a)' % (
      nome_senza_parentesi.replace("'", " ").lower().strip(), name.replace("'", ' ').replace('"', " ").replace("-", " ").strip(), character_id, str(character_description).replace("'", ' ').replace('"', " ").replace("-", " ").strip(),)
  tx.run(query_variant)


In [6]:
driver = GraphDatabase.driver(db_host, auth=("neo4j",password))

In [51]:

for idx, row in tqdm(api_characters.iterrows(), total=api_characters.shape[0]):
    with driver.session() as session:
        try:
            session.write_transaction(
                add_character_node, row['name'], row["id"], row["description"])
        except Exception as e:
            print(e
            )

100%|██████████| 1559/1559 [01:08<00:00, 22.78it/s]
